In [1]:
#!pip install tqdm

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import mal_scraper
import numpy as np

In [3]:
import numpy as np
from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#import pydotplus as pydot 
#import graphviz
from keras.utils import plot_model
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [4]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [5]:
knownscores  = pd.read_csv('mal_scores_train_nonzero_v2.csv')
known_no_scores= pd.read_csv('mal_scores_train_zero.csv')
anime_avgs = np.load('anime_avgs.npy').item()
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict()
review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')
user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [6]:
!nvidia-smi

Fri Sep  8 16:44:45 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   38C    P8    18W / 250W |   7016MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:

n_users = 229756
n_animes = 12826

In [8]:
userid2idx=np.load("user.npy").item()
animeid2idx=np.load("anime.npy").item()
anime_rev_dict=np.load("anime_rev_number.npy").item()


In [9]:
n_highest_rated_shows= 7
k_nearest_shows = 5
t_highest_predicted_score=15
l_max_reviews = 30000

In [10]:
#user_to_rec=input("Mal Username to check") # we ask the wat user we are lookingat

In [11]:
user_to_rec = 'Figgybun'

In [12]:
user_scores = knownscores[knownscores['userid']==user_to_rec]

In [13]:
userratinglist2 = []
if user_scores.shape[0]==0: # This means the user doesn't exist in our dataset and we need to scrape him. 
    reviewdict = {'userid':user_to_rec,'name':user_to_rec,'index':user_to_rec}
    useranime = mal_scraper.get_user_anime_list(user_to_rec)
    if useranime is None:
        print('User Has Not reviewed enough anime') # TODO Add seed shows
        pass
    else:
         for animeiter in range(len(useranime)):
            reviewdict = {'name':user_to_rec,'index':user_to_rec}
            reviewdict['unkey']= str(useranime[animeiter]['id_ref'])+str(user_to_rec)
            statusval = useranime[animeiter]['consumption_status'].value
            #userdict[animeid] = usertest[animenum]['id_ref']
            reviewdict['animescore'] =useranime[animeiter]['score']
            #userdict[animestart] =useranime[animeiter]['start_date']
            #userdict[animefinished] =useranime[animeiter]['finished_date']
            reviewdict['animeid'] =  str(useranime[animeiter]['id_ref'])
            reviewdict['status']= statusval
            userratinglist2.append(reviewdict)
    user_data= pd.DataFrame(userratinglist2) # this is the user's 0 and nonzero scores
    user_data['score']=user_data['animescore'] #standardize the column names
    user_scores=user_data[user_data['score']!=0] # scores
    user_zeroes =user_data[user_data['score']==0] #zero scores
    user_zeroes['animeid']=user_zeroes['animeid'].astype(int)
    useravg= user_scores['score'].mean()
    print(user_scores.head())
    user_scores['score_usr_scaled']=user_scores['score']-useravg
    print(user_scores.head())
    user_scores['animeid']=user_scores['animeid'].astype(int)
    user_scores['animeavg']= user_scores.animeid.map(anime_avgs)
    print(user_scores.head())
    user_scores['score_anime_scaled']=user_scores['score']-user_scores['animeavg']
    print(user_scores.head())
else:
    user_zeroes=known_no_scores[known_no_scores['userid']==user_to_rec]


ERROR:root:Unable to parse the date text "09-26-15" from an anime list
ERROR:root:Unable to parse the date text "04-14-13" from an anime list
ERROR:root:Unable to parse the date text "12-16-14" from an anime list
ERROR:root:Unable to parse the date text "09-29-12" from an anime list
ERROR:root:Unable to parse the date text "07-20-15" from an anime list
ERROR:root:Unable to parse the date text "09-18-13" from an anime list
ERROR:root:Unable to parse the date text "12-21-12" from an anime list
ERROR:root:Unable to parse the date text "09-29-12" from an anime list
ERROR:root:Unable to parse the date text "12-28-15" from an anime list
ERROR:root:Unable to parse the date text "12-21-15" from an anime list
ERROR:root:Unable to parse the date text "06-28-15" from an anime list
ERROR:root:Unable to parse the date text "09-26-13" from an anime list
ERROR:root:Unable to parse the date text "09-28-15" from an anime list
ERROR:root:Unable to parse the date text "09-23-12" from an anime list
ERROR:

   animeid  animescore     index      name     status          unkey  score
10     102           8  Figgybun  Figgybun  COMPLETED    102Figgybun      8
14   15085           9  Figgybun  Figgybun  COMPLETED  15085Figgybun      9
16   16518           9  Figgybun  Figgybun  COMPLETED  16518Figgybun      9
17   12967           8  Figgybun  Figgybun  COMPLETED  12967Figgybun      8
19     104          10  Figgybun  Figgybun  COMPLETED    104Figgybun     10
   animeid  animescore     index      name     status          unkey  score  \
10     102           8  Figgybun  Figgybun  COMPLETED    102Figgybun      8   
14   15085           9  Figgybun  Figgybun  COMPLETED  15085Figgybun      9   
16   16518           9  Figgybun  Figgybun  COMPLETED  16518Figgybun      9   
17   12967           8  Figgybun  Figgybun  COMPLETED  12967Figgybun      8   
19     104          10  Figgybun  Figgybun  COMPLETED    104Figgybun     10   

    score_usr_scaled  
10          0.072289  
14          1.072289  


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
possiblerecs=[]
#user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
seedshows = set(user_scores.sort_values('score_usr_scaled',ascending=False)[:n_highest_rated_shows].animeid.values).union(set(user_scores.sort_values('score_anime_scaled',ascending=False)[:n_highest_rated_shows].animeid.values))

In [15]:
print("seed shows are:")
for rec in seedshows:
    print(anime_names[rec])

seed shows are:
Steins;Gate
Arata Kangatari
Yuu☆Yuu☆Hakusho
Ayashi no Ceres
Free!
Suisei no Gargantia
Sword Art Online
Amnesia
Howl no Ugoku Shiro
Shingeki no Kyojin
Hiiro no Kakera Dai Ni Shou


In [16]:
possiblerecs1=[]
for show in seedshows:
    topn =list(user_review_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
    #print(show)
    possiblerecs1 = possiblerecs1+topn
    #print(type(show))

In [17]:
#len(possiblerecs1)#so now we have one third of the target shows to check. 

In [18]:
#review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [19]:
#review_nlp_numbers.head()

In [20]:
possiblerecs2 = []
for show in seedshows:
    try:
        topk =list(review_nlp_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
        possiblerecs2 = possiblerecs2+topk
    except:
        continue
    #print(type(show))

In [21]:
#review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')

In [22]:
possiblerecs3 = []
k_half = int(k_nearest_shows/2)
for show in seedshows:
    try:
        topk =list(review_lf_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_half+1].index)
        possiblerecs3 = possiblerecs3+topk
    except:
        continue
    #print(type(show))

In [23]:

user_known_anime = list(user_scores.animeid.values)+ list(user_zeroes.animeid.values)

In [24]:
possiblerecs1=[anime for anime in possiblerecs1 if anime not in user_known_anime ]
possiblerecs2=[anime for anime in possiblerecs2 if anime not in user_known_anime ]
possiblerecs3=[anime for anime in possiblerecs3 if anime not in user_known_anime ]
possiblerecs = possiblerecs1+possiblerecs2+possiblerecs3

In [25]:
possiblerecs= list(set(possiblerecs))

In [26]:
possiblerecs= list(set(possiblerecs))

In [27]:
len(possiblerecs)

66

In [28]:
print("Possible recs are:")
for rec in possiblerecs:
    print(anime_names[rec])
    

Possible recs are:
Ibara-Hime matawa Nemuri-Hime
Vampire Knight
Hajime no Ippo
Photokano
Angel Beats!
Naruto
07-Ghost
Suisei no Gargantia: Meguru Kouro, Haruka
Log Horizon 2nd Season
Starry☆Sky
Claymore
Kyoto Animation: Suiei-hen
Kimi ni Todoke
Neon Genesis Evangelion
Hiiro no Kakera: Totsugeki! Tonari no Ikemenzu
Kenpuu Denki Berserk
Hoshi wo Ou Kodomo
Black Lagoon: Roberta's Blood Trail
Inu to Hasami wa Tsukaiyou
Guilty Crown
Zero no Tsukaima
Death Note
Rurouni Kenshin: Meiji Kenkaku Romantan
Dragon Ball Z
Fairy Tail
Amnesia OVA
Shingeki no Kyojin Season 2
Zero no Tsukaima: Futatsuki no Kishi
Tenshi Kinryouku
Boku wa Imouto ni Koi wo Suru
Hunter x Hunter (2011)
Akatsuki no Yona
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Ocha-ken: Chokotto Monogatari
Ao no Exorcist
Death Parade
Hataraku Maou-sama!
Charlotte: Tsuyoimono-tachi
Ookami Kodomo no Ame to Yuki
One Punch Man
Nanatsu no Taizai
High☆Speed!: Free! Starting Days
Rakuen Tsuihou: Expelled from Paradise
Uta no☆Prince-sama♪ Maji Love

In [29]:
print("Possible recs1(User Scores) are:")
for rec in set(possiblerecs1):
    print(anime_names[rec])

Possible recs1(User Scores) are:
Ibara-Hime matawa Nemuri-Hime
Hajime no Ippo
Photokano
Suisei no Gargantia: Meguru Kouro, Haruka
Starry☆Sky
Kyoto Animation: Suiei-hen
Hiiro no Kakera: Totsugeki! Tonari no Ikemenzu
Inu to Hasami wa Tsukaiyou
Rurouni Kenshin: Meiji Kenkaku Romantan
Dragon Ball Z
Fairy Tail
Amnesia OVA
Shingeki no Kyojin Season 2
Tenshi Kinryouku
Hunter x Hunter (2011)
Mirai Nikki (TV)
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Hataraku Maou-sama!
One Punch Man
High☆Speed!: Free! Starting Days
Rakuen Tsuihou: Expelled from Paradise
Uta no☆Prince-sama♪ Maji Love 1000%
Free!: FrFr - Short Movie
Code Geass: Hangyaku no Lelouch R2
Fate/Zero 2nd Season
Dansai Bunri no Crime Edge
Free!: Eternal Summer Special
Steins;Gate: Oukoubakko no Poriomania
Fullmetal Alchemist: Brotherhood
Fushigi Yuugi
Fushigi Yuugi: Eikouden
Fushigi Yuugi OVA 2
Death Note


In [30]:
print("Possible recs2(Tf-IDF) are:")
for rec in set(possiblerecs2):
    print(anime_names[rec])

Possible recs2(Tf-IDF) are:
Vampire Knight
Angel Beats!
Naruto
07-Ghost
Kimi ni Todoke
Neon Genesis Evangelion
Hoshi wo Ou Kodomo
Kenpuu Denki Berserk
Guilty Crown
Dragon Ball Z
Hunter x Hunter (2011)
Akatsuki no Yona
Ao no Exorcist
Ookami Kodomo no Ame to Yuki
Nanatsu no Taizai
Koutetsujou no Kabaneri
Uta no☆Prince-sama♪ Maji Love 1000%
Durarara!!
Haikyuu!!
Eureka Seven
Accel World
Fushigi Yuugi


In [31]:
print("Possible recs3(Hidden Factors) are:")
for rec in set(possiblerecs3):
    print(anime_names[rec])

Possible recs3(Hidden Factors) are:
Charlotte: Tsuyoimono-tachi
Elfen Lied
Shakugan no Shana II (Second)
Bible Black: Imari Ryoujoku Genba
Black Lagoon: Roberta's Blood Trail
Ocha-ken: Chokotto Monogatari
Zero no Tsukaima
Oni Chichi
Zero no Tsukaima: Futatsuki no Kishi
Boku wa Imouto ni Koi wo Suru
High School DxD BorN
Karin
Log Horizon 2nd Season
Claymore
Death Parade


In [32]:
ratings_score_test = pd.read_csv('mal_scores_test_nonzero_v2.csv')

In [33]:
ratings_score_test.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled,user_rev_count,anime_rev_count
0,0,0,13,13,889,9,SeinfeldHentai,SeinfeldHentai,COMPLETED,889SeinfeldHentai,SeinfeldHentai,9,9.111111,8.157417,0.842583,-0.111111,9,18397
1,1,1,16,16,28907,9,SeinfeldHentai,SeinfeldHentai,COMPLETED,28907SeinfeldHentai,SeinfeldHentai,9,9.111111,7.894818,1.105182,-0.111111,9,15497
2,2,2,233,233,21185,7,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,21185OPERATIONiDROID,OPERATIONiDROID,7,7.767857,7.921898,-0.921898,-0.767857,56,3329
3,3,3,243,243,18507,9,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,18507OPERATIONiDROID,OPERATIONiDROID,9,7.767857,7.631896,1.368104,1.232143,56,16786
4,4,4,248,248,15059,8,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,15059OPERATIONiDROID,OPERATIONiDROID,8,7.767857,8.219309,-0.219309,0.232143,56,1564


In [34]:
recs=pd.DataFrame(possiblerecs, columns=['animeid'])

In [35]:
recs['userid']=user_to_rec

In [36]:
recs['animescore']=0
recs['score_anime_scaled']=0
recs['score_usr_scaled']=0


In [37]:
#recs.head()

In [38]:
#type(userid2idx)

In [39]:
#userid2idx

In [40]:
#user_scores

In [41]:
user_scores['userid']= user_scores['name']
user_zeroes['userid']= user_zeroes['name']
user_scores = user_scores[['animeid','userid','status']]
user_zeroes = user_zeroes[['animeid','userid','status']]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
recs['user_id_emb'] = len(userid2idx)+2
concatlist = [user_scores,user_zeroes]
user_scores = pd.concat(concatlist,ignore_index=True)

In [43]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
user_scores['user_id_emb']= len(userid2idx)+1

In [44]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
recs['anime_id_emb'] = recs.animeid.apply(lambda x: animeid2idx[x])



In [45]:
user_scores['target']=user_scores['status']=="COMPLETED"
user_scores['target']=user_scores['target'].astype(int)


In [46]:
n_factors =36

In [47]:
def embedding_input_anime1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in1, u1 = embedding_input_user1('user_id_in', n_users+1221, n_factors, 0)
u1.trainable=True
anime_in1, a1 = embedding_input_anime1('anime_id_in', n_animes+47, n_factors, 0)
a1.trainable=False

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
# nn1, It is given no modifications for status. It is predicting score
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dropout(0.1, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions',activation='sigmoid')(x)
x.trainable=False
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [49]:
nn1.load_weights('nn_score_weights_pred.h5',by_name=True)

In [50]:
!nvidia-smi

Fri Sep  8 16:44:55 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   39C    P2    59W / 250W |   7052MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [51]:
nn1.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.target, epochs=18)

Epoch 1/18
186/186 [==============================] - 0s - loss: 0.2248     
Epoch 2/18
186/186 [==============================] - 0s - loss: 0.2093     
Epoch 3/18
186/186 [==============================] - 0s - loss: 0.1879     
Epoch 4/18
186/186 [==============================] - 0s - loss: 0.1787     
Epoch 5/18
186/186 [==============================] - 0s - loss: 0.1785     
Epoch 6/18
186/186 [==============================] - 0s - loss: 0.1626     
Epoch 7/18
186/186 [==============================] - 0s - loss: 0.1559     
Epoch 8/18
186/186 [==============================] - 0s - loss: 0.1478     
Epoch 9/18
186/186 [==============================] - 0s - loss: 0.1436     
Epoch 10/18
186/186 [==============================] - 0s - loss: 0.1276     
Epoch 11/18
186/186 [==============================] - 0s - loss: 0.1317     
Epoch 12/18
186/186 [==============================] - 0s - loss: 0.1148     
Epoch 13/18
186/186 [==============================] - 0s - loss: 0.1102 

In [52]:
recs['score']=nn1.predict([recs.user_id_emb, recs.anime_id_emb])

In [53]:
#recs.head()

In [54]:
#recs.sort_values('score',ascending=False)

In [55]:
recs['anime_rev_count']= recs.animeid.map(anime_rev_dict)

In [56]:
recs=recs[recs['anime_rev_count']<l_max_reviews]

In [57]:
#recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)

In [58]:
#recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

In [59]:
#recs.sort_values('score',ascending=False).head(t_highest_predicted_score)

In [60]:
#score_user_preds =recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)
score_preds =recs.sort_values('score',ascending=False).head(t_highest_predicted_score)
#score_anime_preds =recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

In [61]:
#score_preds.animeid.values

In [62]:
print("seed shows are:")
for rec in seedshows:
    print(anime_names[rec])
print(" ")
print("Possible recs1(User Scores) are:")
for rec in set(possiblerecs1):
    print(anime_names[rec])
print(' ' )
print("Possible recs2(Tf-IDF) are:")
for rec in set(possiblerecs2):
    print(anime_names[rec])
print(" ")
print("Possible recs3(Hidden Factors) are:")
for rec in set(possiblerecs3):
    print(anime_names[rec])

seed shows are:
Steins;Gate
Arata Kangatari
Yuu☆Yuu☆Hakusho
Ayashi no Ceres
Free!
Suisei no Gargantia
Sword Art Online
Amnesia
Howl no Ugoku Shiro
Shingeki no Kyojin
Hiiro no Kakera Dai Ni Shou
 
Possible recs1(User Scores) are:
Ibara-Hime matawa Nemuri-Hime
Hajime no Ippo
Photokano
Suisei no Gargantia: Meguru Kouro, Haruka
Starry☆Sky
Kyoto Animation: Suiei-hen
Hiiro no Kakera: Totsugeki! Tonari no Ikemenzu
Inu to Hasami wa Tsukaiyou
Rurouni Kenshin: Meiji Kenkaku Romantan
Dragon Ball Z
Fairy Tail
Amnesia OVA
Shingeki no Kyojin Season 2
Tenshi Kinryouku
Hunter x Hunter (2011)
Mirai Nikki (TV)
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Hataraku Maou-sama!
One Punch Man
High☆Speed!: Free! Starting Days
Rakuen Tsuihou: Expelled from Paradise
Uta no☆Prince-sama♪ Maji Love 1000%
Free!: FrFr - Short Movie
Code Geass: Hangyaku no Lelouch R2
Fate/Zero 2nd Season
Dansai Bunri no Crime Edge
Free!: Eternal Summer Special
Steins;Gate: Oukoubakko no Poriomania
Fullmetal Alchemist: Brotherhood
Fushi

In [63]:
print("Recommendations for likely to watch  for {0} are:".format(user_to_rec))
for rec in score_preds.animeid.values:
    print(anime_names[rec])

Recommendations for likely to watch  for Figgybun are:
Kyoto Animation: Suiei-hen
Zero no Tsukaima: Futatsuki no Kishi
Boku wa Imouto ni Koi wo Suru
Vampire Knight
Dragon Ball Z
Ookami Kodomo no Ame to Yuki
Steins;Gate: Oukoubakko no Poriomania
Nanatsu no Taizai
Karin
Amnesia OVA
Zero no Tsukaima
Kimi ni Todoke
Tenshi Kinryouku
Hataraku Maou-sama!
Claymore


In [64]:
!nvidia-smi

Fri Sep  8 16:44:58 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   41C    P2   107W / 250W |   7156MiB / 11170MiB |     46%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  